In [11]:
import requests
import re
import urllib
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from datetime import datetime

In [12]:
def get_url(title, location, page):
    template_url = 'https://www.indeed.com/jobs?q={}&l={}&fromage=7&start={}'
    url = template_url.format(urllib.parse.quote(title), urllib.parse.quote(location), page)
    return url

In [13]:
def get_records(card):

    try:
        job_title = card.find('h2', {'class':'jobTitle'}).findAll('span')[-1].text

        if 'data scien' not in job_title.lower():
            return None
    except:
        job_title = ''
    
    try:
        job_conpany = card.find('span', {'class':'companyName'}).text
    except:
        job_conpany = ''

    try:
        job_rating = card.find('span', {'class':'ratingsDisplay withRatingLink'}).text
    except:
        job_rating = ''
    
    try:
        job_location = [i.text for i in card.find_all('div', {'class':'companyLocation'})]
    except:
        job_location = ''
    
    try:
        job_salary =  card.find('span', {'class':'salary-snippet'}).text
    except:
        job_salary = ''
    
    try:
        job_date = card.find('span', {'class':'date'}).text
    except:
        job_date = ''

    extract_date = datetime.today().strftime('%Y-%m-%d')

    job_url = 'https://www.indeed.com'+ card['href']
    return job_title, job_conpany, job_rating, job_location, job_salary, job_date, extract_date, job_url


In [14]:
def get_info(title, location):
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"}
    records = []
    details = []
    no_of_page = 0
    # for i in range(0, 1000, 10):
    i = 0
    while True:
        if i % 100 == 0 :
            wait = randint(20, 40)
            sleep(wait)
        url = get_url(title, location, i*10)
        print(url)
        res = requests.get(url, header)
        soup = BeautifulSoup(res.text, 'html.parser')

        #get total jobs
        if no_of_page == 0:
            try:
                no_of_page_string = soup.find(id='searchCountPages').text.strip()
                print(no_of_page_string)
                no_of_page = int(re.sub('\W+', '', no_of_page_string.split()[3]))
            except:
                print('No total number')
                pass

        try:
            cards = soup.find(id='mosaic-provider-jobcards').findChildren("a" , recursive=False)
        except:
            print('No job list')
            break

        for card in cards:
            record = get_records(card)
            if record:
                records.append(record)

        wait = randint(30, 40)
        sleep(wait)

        i += 1
        # res.close()
    return records, details, no_of_page*15

In [15]:
def saveDate(title, records, details):
    col = ['Title', 'Company', 'Rating', 'Location', 'Salary', 'Post_date', 'Extract_date', 'Job_url']
    df_basic = pd.DataFrame(records)
    df_basic.columns = col
    df_detail = pd.DataFrame.from_dict(details)
    df = pd.concat([df_basic, df_detail], axis=1)
    df.to_csv(f'{title}.csv')

In [16]:
no_of_jobs = {}

#get records from 'Data scientist' in 'UniedStates'
records, details, total = get_info('Data scientist', 'United states')
try:
    saveDate(f'Data scientist no detail', records, details)
except:
    print("No data stored")
no_of_jobs['Data scientist'] = total


https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=7&start=0
Page 1 of 3,906 jobs
https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=7&start=10
https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=7&start=20
https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=7&start=30
https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=7&start=40
https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=7&start=50
https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=7&start=60
https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=7&start=70
https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=7&start=80
https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=7&start=90
https://www.indeed.com/jobs?q=Data%20scientist&l=United%20states&fromage=7&start=100
https://www.indeed.com/jobs?q=Data%20scientist&l=United

In [17]:
no_of_jobs

{'Data scientist': 58590}

In [18]:
len(records)

4391

In [19]:
df_basic = pd.DataFrame(records, columns=['Title', 'Company', 'Rating', 'Location', 'Salary', 'Post_date', 'Extract_date', 'Job_url'])
df_basic

,Title,Company,Rating,Location,Salary,Post_date,Extract_date,Job_url
0,Senior Data Scientist,CyberCoders,3.4,"[Lake Forest, CA 92630+2 locations]",,1 day ago,2021-09-18,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Principal Data Scientist,DISYS,3.5,"[Colorado Springs, CO 80909 (East Colorado Spr...",$65 - $75 an hour,Today,2021-09-18,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,Data Scientist,Insight Global,3.9,[+1 locationRemote],$65 - $75 an hour,3 days ago,2021-09-18,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
3,Data Scientist,Pri Global Inc,,"[San Francisco, CA•Remote]",$50 - $55 an hour,3 days ago,2021-09-18,https://www.indeed.com/company/Pri-Global-Inc/...
4,Software Engineer - Data Scientist,RefleXion Medical,3.5,"[Hayward, CA 94545 (Glen Eden area)]",,Today,2021-09-18,https://www.indeed.com/rc/clk?jk=56c53f54d6307...
...,...,...,...,...,...,...,...,...
4386,Technical Team Member – Distributed Data Scien...,Agnik,,"[Berkeley, CA]",,7 days ago,2021-09-18,https://www.indeed.com/rc/clk?jk=cad2933840fed...
4387,SENIOR DATA SCIENTIST (GLOBAL COMMERCIAL ANALY...,NIKE INC,4.1,"[Brush Prairie, WA 98606+4 locations]",,2 days ago,2021-09-18,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
4388,"Data Scientist - Richmond, Virginia (VA)",LL Flooring,3.2,"[Richmond, VA 23230]",,7 days ago,2021-09-18,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
4389,Data Scientist,Applied Information Sciences,4.1,"[Aberdeen Proving Ground, MD 21005•Remote]",,3 days ago,2021-09-18,https://www.indeed.com/rc/clk?jk=4f7fb72171981...
